# Intro: Feature Map and How to Map it to Original Image
very nice demonstration of how regions in feature map, map into the original image. 
The idea is used in both YOLO and faster RCNN

![lectur1](./images/snap_lecture_1.png)
![lectur2](./images/snap_lecture_2.png)
![lectur3](./images/snap_lecture_3.png)
![lectur4](./images/snap_lecture_4.png)

# YOLO TRAINING

One training data is generated per grid. Each training label consists of 8 values
### 1) first element pc represents the probability of an object of interest being in the grid
### 2) the next 3 are class probabilities
### 3) the next 4 are bounding box coordinate of the `entire` object. (x, y) represent the centroid of the human in relative terms (values between 0,1) and (w, h) represent the corners of the bounding box in relative terms (h = 0.5 means height is 1/2 of the entire image height, and w = 0.2 means the wdith is 1/5 of the input image width). 

we need to create this vector for each gird. Notice only one of the 3 grids for this human in the image gets pc = 1 the rest are 0. only the grid with none 0 pc gets the true coordinate of the human, the remaining 2 grids will get 0 values for the entire vector (or may be they still get some values for coordinates x,y,w,h i do not know. in the image below we will get 10x7 = 70 label vectors of length 8

![training_1](./images/snap_lecture_yolo1.png)


here are values for the center grid. notice relative/normalized values of x, y, which are between 0, 1. Height for this grid is 0.4 and width is 0.1, each referring to value relative to the hieght and width of the entire image. This ensures the gird labels coordinate can extend beyond the boundaries of the grid covering the highlighted yello area and are between 0, 1 (normaized values are easier to train)




![training_4](./images/snap_lecture_yolo4.png)
![training_5](./images/snap_lecture_yolo5.png)

look at the middle grid containing huamn, the only among three that will get non zero pc and c1. I am not sure what values all these 3 grid cells get for x,y,w,h


![training_2](./images/snap_lecture_yolo2.png)

now lets put all the training labels for training a CNN model together. the pink grid and its training 8 dimensional label is shown. 

![training_3](./images/snap_lecture_yolo3.png)

# NonMaximal Supression
This model has an issue. It can assign several grids to the same object whose coordinates closely overlap. which we solve by NMS
it achieves 2 objectives:
#### 1) all grids with low pc values are removed from consideration
#### 2) those with large pc that overlap significantly are consolidated into one bounding box based on the grid with largest pc (confidence)
![training_6](./images/snap_lecture_yolo6.png)

## Let us define IOU: 
High value of IOU between a true object bounding box and predicted region by the grid is desirable. 
This value between prediction and true label can only be calculated during training

![training_7](./images/snap_lecture_yolo7.png)


## Let's use IOU for NMS during Prediction to remove overlapping prediction boxes
remember during training, we used this same image above. only  1 grid in the training had pc = 1 and the other 2 are 0. Now imagine YOLO has not seen this image in training, but we give it this image for predictio. 
example below shows (x,y,w,h) coordinates from prediction results from the three grids in the image above. You can see tht all 3 grids can be predicting the human object despite how we labeled our training data. We need to find a way to consolidate these prediction using NMS.

Research true algorithm of NMS : here is what i think, we sort all anchor boxes with pc > threshold. We start from the highest one. We calculate its IOU with rest of them anchord boxes. We remove the ones with which it has a high IOU. Then we go to the next highest pc that we have not processed yet, and repeat the same process again. Here we have 3 pc values of 0.9, 0.8 and 0.7 with high IOU between them. In this case we remove 0.8 and 0.7 prediction in favor of 0.9

![training_8](./images/snap_lecture_yolo8.png)

# Anchor Boxes
what if you have multiple objects on top of each other in an image. We can use instead of a 8 dimensional label/output a 16 dimensional one to allow the ability of encoding 2 objects in every grid. This allows us to extend detection over grid to upto 2 objects. 
we can have two different form factors for our anchor boxes, the fat and long one for example is shown below. I assume by providing labels in the form of fat and long and being consistent, the model trains and learn to only consider cars in the first 8 elements of the output and humans in the second one. 
the shortcomings of this method: what if there are 2 humans in a grid, what if there are 3 objects in a grid

![training_9](./images/snap_lecture_yolo9.png)
![training_10](./images/snap_lecture_yolo10.png)